In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import stats
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from ipywidgets import interact
from sklearn.model_selection import GridSearchCV

load and read the data

In [ ]:
data="imports-85.data"
df = pd.read_csv(data,header = None)

Add the header for each column

In [ ]:
headers = ["symboling","normalized-losses","make","fuel-type","aspiration","num-of-doors","body-style","drive-wheels",
           "engine-location","wheel-base","length","width","height","curb-weight","engine-type","num-of-cylinders",
           "engine-size","fuel-system","bore","stroke","compression-ratio","horsepower","peak-rpm","city-mpg","highway-mpg",
           "price"]
df.columns = headers

Drop/ Remove rows that contains missing values like NaN

In [ ]:
df.dropna(subset = ["price"], axis=0, inplace=True)
df.dropna(subset = ["peak-rpm"], axis=0, inplace=True)

Convert datatype from object to float

In [ ]:
vals_price = pd.to_numeric(df["price"],errors='coerce')
df["price"] = vals_price.fillna(vals_price.mean())

vals_peakRpm = pd.to_numeric(df["peak-rpm"],errors='coerce')
df["peak-rpm"] = vals_peakRpm.fillna(vals_peakRpm.mean())

vals_hp = pd.to_numeric(df["horsepower"],errors='coerce')
df["horsepower"] = vals_hp.fillna(vals_hp.mean())

vals = pd.to_numeric(df["normalized-losses"],errors='coerce')
df["normalized-losses"] = vals.fillna(vals.mean())

Convert mpg to L/100kms and rename the column

In [ ]:
df["city-mpg"] = 235/df["city-mpg"]
df.rename(columns={"city_mpg": "city-L/100km"}, inplace=True)

In [ ]:
df["length"] = (df["length"]-df["length"].mean())/df["length"].std()
df["width"] = (df["width"]-df["width"].mean())/df["width"].std()
df["height"] = (df["height"]-df["height"].mean())/df["height"].std()

In [ ]:
bins = np.linspace(min(df["price"]),max(df["price"]),4)
group_names = ["Low", "Medium","High"]
df["price-binned"] = pd.cut(df["price"],bins,labels=group_names,include_lowest=True)

Convert categorical values to quantitative values

In [ ]:
pd.get_dummies(df['fuel-type'])

summarize the categorical data using value_counts()

In [ ]:
drive_wheels_counts=df["drive-wheels"].value_counts()
drive_wheels_counts.rename(columns={"drive-wheels":"value_counts"}, inplace=True)
drive_wheels_counts.index.name = 'drive-wheels'
print(drive_wheels_counts)

boxplot example

In [ ]:
sns.boxplot(x="drive-wheels",y="price",data=df)

Each observation in a scatter plot is represented as a point. This plot shows the relationship between two variables:
The predictor variable: is the variable that you are using to predict an outcome. In this
case, our predictor variable is the engine size.
The target variable: is the variable that you are trying to predict. In this case, our
target variable is the price, since this would be the outcome.
In this case, we will thus plot the engine size on the x-axis and the price on the y-axis.
We are using the Matplotlib function “scatter” here, taking in x and a y variable.

In [ ]:
y=df["engine-size"]
x=df["price"]
plt.scatter(x,y)
plt.title("Scatterplot of Engine Size vs Price")
plt.xlabel("Engine Size")
plt.ylabel("Price")

By using groupby to find the average price of vehicles and
observe how they differ between different types of “body styles” and “drive wheels”
variables.
To do this, we first pick out the three data columns we are interested in, which is done
in the first line of code.
We then group the reduced data according to ‘drive wheels’ and ‘body style’ in
the second line.
Since we are interested in knowing how the average price differs across the board, we
can take the mean of each group and append this bit at the very end of line 2. 
We can see that, according to our data, rear wheel drive convertibles and rear wheel drive hardtops have the highest value, while four wheel drive hatchbacks have the lowest value.

In [ ]:
df_test = df[['drive-wheels','body-style','price']]
df_grp = df_test.groupby(['drive-wheels','body-style'], as_index=False).mean()
df_grp

We can transform the above table to a pivot table by using the pivot
method.
In the previous table, both ‘drive wheels’ and ‘body style’ were listed in columns.
A pivot table has one variable displayed along the columns and the other variable displayed
along the rows.
Just with one line of code and by using the pandas pivot method, we can pivot the “body
style” variable so it is displayed along the columns and the “drive wheels” will
be displayed along the rows.
The price data now becomes a rectangular grid, which is easier to visualize.

In [ ]:
df_pivot = df_grp.pivot(index='drive-wheels', columns='body-style')
df_pivot

Heat map takes a rectangular grid of data and assigns a color intensity based on the
data value at the grid points.
It is a great way to plot the target variable over multiple variables and through this get
visual clues of the relationship between these variables and the target.

In [ ]:
plt.pcolor(df_pivot, cmap='RdBu')
plt.colorbar()
plt.show()

Analysis of Variance (ANOVA)
In the first line we extract the make and price data.
Then, we'll group the data by different makes.
The ANOVA test can be performed in Python using the f_oneway method as the built-in
function of the Scipy package.
We pass in the price data of the two car make groups that we want to compare and it calculates
the ANOVA results.
The prices between Hondas and Subarus are not significantly different, as the F-test
score is less than 1 and p-value is larger than 0.05.
We can do the same for Honda and Jaguar.
The prices between Hondas and Jaguars are significantly different, since the F-score
is very large (F = 401) and the p-value is larger than 0.05.
All in all, we can say that there is a strong correlation between a categorical variable
and other variables, if the ANOVA test gives us a large F-test value and a small p-value.

In [ ]:
df_anova = df[["make", "price"]]
grouped_anova = df_anova.groupby(["make"])
anova_results_1 = stats.f_oneway(grouped_anova.get_group("honda")["price"], grouped_anova.get_group("subaru")["price"])
anova_results_2 = stats.f_oneway(grouped_anova.get_group("honda")["price"], grouped_anova.get_group("jaguar")["price"])
print(anova_results_1)
print(anova_results_2)

Regression Plot provides good estimate of:

1. The relationship betweeen two variables.
2. The strength of the correlation
3. The direction of the relationship (positive or negative)

Let's look at the correlation between engine size and price. This time we'll visualize these two variables using
a scatter plot and an added linear line called a regression line, which indicates the relationship between the two.
The main goal of this plot is to see whether the engine size has any impact on the price.
In this example, you can see that the straight line through the data points is very
steep which shows that there's a positive linear relationship between the two variables.
With increase in values of engine size, values of price go up as well and the slope of the line is positive. So there is a positive correlation between engine size and price.

In [ ]:
sns.regplot(x="engine-size", y="price", data = df)
plt.ylim(0,)

As another example, now let's look at the relationship between highway miles per gallon to see its impact on the car price. As we can see in this plot, when highway miles per gallon value goes up the value price goes down. Therefore there is a negative linear relationship between highway miles per gallon and price. Although this relationship is negative the slope of the line is steep which means that the highway miles per gallon is still a good predictor of price. These two variables are said to have a negative correlation.

In [ ]:
sns.regplot(x="highway-mpg", y="price", data = df)
plt.ylim(0,)

Finally, we have an example of a weak correlation. For example, both low peak RPM and high values of peak RPM have low and high prices. Therefore, we cannot use RPM to predict the values.

In [ ]:
sns.regplot(x="peak-rpm", y="price", data = df)
plt.ylim(0,)

We use the “residplot” function. The first parameter is a series of dependent variable or feature. The second parameter is a Series of dependent variable or target. We see in this case the Residuals have a curvature.

In [ ]:
sns.residplot(df['highway-mpg'], df['price'])

Pearson correlation method will gives two values: the correlation coefficient and the P-value.For the correlation coefficient,
a value close to 1 implies a large positive correlation, while a value close to negative 1 implies a large negative correlation, and a value close to zero implies no correlation between the variables. Next, the P-value will tell us how
certain we are about the correlation that we calculated. For the P-value, a value less than .001 gives us a strong certainty about the correlation coefficient that we calculated. A value between.001 and.05 gives us moderate certainty. A value between.05 and.1 will give us a weak certainty. And a P-value larger than.1 will give us no certainty of correlation at all.
We can say that there is a strong correlation when the correlation coefficient is close to 1 or negative 1, and the P-value is less than.001. The following plot shows data with different correlation values. In this example, we want to look at the correlation between the variable's horsepower and car price. We can see that the correlation coefficient is approximately.8, and this is close to 1. So there is a strong positive correlation. We can also see that the P-value is very small, much smaller than.001. And so we can conclude that we are certain about the strong positive correlation. Taking all variables into account,
we can now create a heatmap that indicates the correlation between each of the variables with one another. The color scheme indicates the Pearson correlation coefficient, indicating the strength of the correlation between two variables. We can see a diagonal line with a dark red color, indicating that all the values on this diagonal are highly correlated. This makes sense because when you look closer, the values on the diagonal are the correlation of all variables with themselves, which will be always 1. This correlation heatmap gives us a good overview of how the different variables are related to one another and,
most importantly, how these variables are related to price.


In [ ]:
pearson_coef, p_value = stats.pearsonr(df["horsepower"], df["price"])
print(pearson_coef)
print(p_value)

Create a Linear Regression Object using the constructor

In [ ]:
lm = LinearRegression()

Define the predictor variable and the target variable

In [ ]:
X = df['highway-mpg']
Y = df['price']
X1 = df['highway-mpg']
Y1 = df['price']
X = X.values.reshape(-1,1)
Y = Y.values.reshape(-1,1)

Use lm.fit(X,Y) to fit the model, i.e. fine the parameters b0 and b1

In [ ]:
lm.fit(X,Y)

In [ ]:
Yhat = lm.predict(X)

Intercept b0 of a linear regression

In [ ]:
lm.intercept_

Slope b1 of a linear regression

In [ ]:
lm.coef_

A distribution plot counts the predicted value versus the actual value. These plots are extremely useful for visualizing models with more than one independent variable or feature. We examine the vertical axis.
- We then count and plot the number of predicted points that are approximately equal to one.
- We then count and plot the number of predicted points that are approximately equal to two.
- We repeat the process for predicted points that are approximately equal to three.
Then we repeat the process for the target values.
In this case, all the target values are approximately equal to two. The values of the targets and predicted values are continuous. A histogram is for discrete values. Therefore pandas will convert them to a distribution.
The vertical access is scaled to make the area under the distribution equal to one.
This is an example of using a distribution plot.
The dependent variable or feature is price.
The fitted values that result from the model are in blue.
The actual values are in red.
We see the predicted values for prices in the range from $40 000 to $50 000 are inaccurate.
The prices in the region form $10 000 to $20 000 are much closer to the target value.

In [ ]:
ax1 = sns.distplot(df['price'], hist=False, color = "r", label="Actual Value")
sns.distplot(Yhat, hist=False, color = "b", label = "Fitted Values", ax= ax1)

Calculating the R-squared value by using the score() method, in the linear regression object. From the value that we get from the below example, we can say that approximately 47.68% of the variation of the price is explained by this simple linear model. The R-square value is usually between 0 and 1, if the value is a negative number then it can be due to overfitting.

In [ ]:
X2 = df['highway-mpg'].values.reshape(-1,1)
Y2 = df['price'].values.reshape(-1,1)
linReg = LinearRegression()
linReg.fit(X2, Y2)
linReg.score(X2, Y2,sample_weight=None)

Now we want to find out what the price would be for a car that has a highway-mpg of 30. Plugging this value into the predict() method, gives us a resulting price of $13,799.86. This seems to make sense, for example, the value is not negative, extremely high or extremely low.

In [ ]:
X3 = df['highway-mpg'].values.reshape(-1,1)
Y3 = df['price'].values.reshape(-1,1)
linReg = LinearRegression()
linReg.fit(X3, Y3)
linReg.predict([[30]])

<h2>Model Evaluation</h2>

A popular function in the sci-kit learn package for splitting datasets is the "train test split" function. This function randomly splits a dataset into training and testing subsets. The output is an array: 
 * "x_train" and "y_train", the subsets for training.
 * "x_test" and "y_test",the subsets for testing. 
<br>In this case, the "test size" percentage of the data for the testing set. Here it is 15%. The random state is a random seed for random dataset splitting.

<h3>Functions for plotting</h3>

In [ ]:
def DistributionPlot(RedFunction, BlueFunction, RedName, BlueName, Title):
    width = 12
    height = 10
    plt.figure(figsize=(width, height))

    ax1 = sns.distplot(RedFunction, hist=False, color="r", label=RedName)
    ax2 = sns.distplot(BlueFunction, hist=False, color="b", label=BlueName, ax=ax1)

    plt.title(Title)
    plt.xlabel('Price (in dollars)')
    plt.ylabel('Proportion of Cars')

    plt.show()
    plt.close()

In [ ]:
def PollyPlot(xtrain, xtest, y_train, y_test, lr,poly_transform):
    width = 12
    height = 10
    plt.figure(figsize=(width, height))
    
    
    #training data 
    #testing data 
    # lr:  linear regression object 
    #poly_transform:  polynomial transformation object 
 
    xmax=max([xtrain.values.max(), xtest.values.max()])

    xmin=min([xtrain.values.min(), xtest.values.min()])

    x=np.arange(xmin, xmax, 0.1)


    plt.plot(xtrain, y_train, 'ro', label='Training Data')
    plt.plot(xtest, y_test, 'go', label='Test Data')
    plt.plot(x, lr.predict(poly_transform.fit_transform(x.reshape(-1, 1))), label='Predicted Function')
    plt.ylim([-10000, 60000])
    plt.ylabel('Price')
    plt.legend()

In [ ]:
y_data = df['price']
x_data = df.drop('price', axis=1)

Split data into training and testing data using the function train_test_split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.15, random_state=1)
print("number of test samples :", x_test.shape[0])
print("number of training samples:",x_train.shape[0])

We create a Linear Regression object:

In [ ]:
lre = LinearRegression()

we fit the model using the feature horsepower

In [ ]:
lre.fit(x_train[['horsepower']], y_train)

Let's Calculate the R^2 on the test data:

In [ ]:
lre.score(x_train[['horsepower']], y_train)

Sometimes you do not have sufficient testing data; as a result, you may want to perform Cross-validation. Let's go over several methods that you can use for Cross-validation. <br> We input the object, the feature in this case ' horsepower', the target data (y_data). The parameter 'cv' determines the number of folds; in this case 4.

In [ ]:
Rcross = cross_val_score(lre, x_data[['horsepower']], y_data, cv=4)

The default scoring is R^2; each element in the array has the average R^2 value in the fold:

In [ ]:
Rcross

We can calculate the average and standard deviation of our estimate:

In [ ]:
print("The mean of the folds are", Rcross.mean(), " and the standard deviation is", Rcross.std())

'cross_val_predict' function splits up the data into the specified number of folds, using one fold to get a prediction while the rest of the folds are used as test data. <br> We input the object, the feature in this case 'horsepower' , the target data y_data. The parameter 'cv' determines the number of folds; in this case 4. We can produce an output:

In [ ]:
yhat = cross_val_predict(lre, x_data[['horsepower']], y_data, cv=4)
yhat[0:5]

<h3>Overfitting, Underfitting and Model Selection</h3>

<p>It turns out that the test data sometimes referred to as the out of sample data is a much better measure of how well your model performs in the real world.  One reason for this is overfitting; It turns out these differences are more apparent in Multiple Linear Regression and Polynomial Regression so we will explore overfitting in that context.</p>

Let's create Multiple linear regression objects and train the model using 'horsepower', 'curb-weight', 'engine-size' and 'highway-mpg' as features.

In [ ]:
lr = LinearRegression()
lr.fit(x_train[['horsepower', 'curb-weight', 'engine-size', 'highway-mpg']], y_train)

Prediction using training data:

In [ ]:
yhat_train = lr.predict(x_train[['horsepower', 'curb-weight', 'engine-size', 'highway-mpg']])
yhat_train[0:5]

Prediction using test data:

In [ ]:
yhat_test = lr.predict(x_test[['horsepower', 'curb-weight', 'engine-size', 'highway-mpg']])
yhat_test[0:5]

Performing some model evaluation using our training and testing data separately.

In [ ]:
Title = 'Distribution  Plot of  Predicted Value Using Training Data vs Training Data Distribution'
DistributionPlot(y_train, yhat_train, "Actual Values (Train)", "Predicted Values (Train)", Title)

In [ ]:
Title='Distribution  Plot of  Predicted Value Using Test Data vs Data Distribution of Test Data'
DistributionPlot(y_test,yhat_test,"Actual Values (Test)","Predicted Values (Test)",Title)

Comparing Figure 1 and Figure 2; it is evident the distribution of the test data in Figure 1 is much better at fitting the data. This difference in Figure 2 is apparent where the ranges are from 5000 to 15 000. This is where the distribution shape is exceptionally different.<br> Let's perform polynomial regression to check whether it exhibits a drop in the prediction accuracy when analysing the test dataset.

<h4>Overfitting</h4>
<p>Overfitting occurs when the model fits the noise, not the underlying process. Therefore when testing your model using the test-set, your model does not perform as well as it is modelling noise, not the underlying process that generated the relationship. Let's create a degree 5 polynomial model.</p>

Let's use 55 percent of the data for testing and the rest for training:

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.45, random_state=0)

We will perform a degree 5 polynomial transformation on the feature 'horse power'.

In [ ]:
pr = PolynomialFeatures(degree=5)
x_train_pr = pr.fit_transform(x_train[['horsepower']])
x_test_pr = pr.fit_transform(x_test[['horsepower']])
pr

Now let's create a linear regression model "poly" and train it.

In [ ]:
poly = LinearRegression()
poly.fit(x_train_pr, y_train)

We can see the output of our model using the method "predict." then assign the values to "yhat".

In [ ]:
yhat = poly.predict(x_test_pr)
yhat[0:5]

Let's take the first five predicted values and compare it to the actual targets.

In [ ]:
print("Predicted values:", yhat[0:4])
print("True values:", y_test[0:4].values)

We will use the function "PollyPlot" that we defined at the beginning of the lab to display the training data, testing data, and the predicted function.

In [ ]:
PollyPlot(x_train[['horsepower']], x_test[['horsepower']], y_train, y_test, poly,pr)

A polynomial regression model, red dots represent training data, green dots represent test data, and the blue line represents the model prediction. We see that the estimated function appears to track the data

In [ ]:
def f(order, test_data):
    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=test_data, random_state=0)
    pr = PolynomialFeatures(degree=order)
    x_train_pr = pr.fit_transform(x_train[['horsepower']])
    x_test_pr = pr.fit_transform(x_test[['horsepower']])
    poly = LinearRegression()
    poly.fit(x_train_pr,y_train)
    PollyPlot(x_train[['horsepower']], x_test[['horsepower']], y_train,y_test, poly, pr)

The following interface allows you to experiment with different polynomial orders and different amounts of data.

In [ ]:
interact(f, order=(0, 6, 1), test_data=(0.05, 0.95, 0.05))

<h3>Ridge regression</h3> <br>
Using Ridge Regression we will check how the parameter alpha changes the model. <br>
Let's perform a degree two polynomial transformation on our data.

In [ ]:
pr = PolynomialFeatures(degree=2)
x_train_pr = pr.fit_transform(x_train[['horsepower', 'curb-weight', 'engine-size', 'highway-mpg','normalized-losses','symboling']])
x_test_pr=pr.fit_transform(x_test[['horsepower', 'curb-weight', 'engine-size', 'highway-mpg','normalized-losses','symboling']])

Let's create a Ridge regression object, setting the regularization parameter to 0.1

In [ ]:
RigeModel=Ridge(alpha=0.1)

Like regular regression, we can fit the model using the method fit.

In [ ]:
RigeModel.fit(x_train_pr, y_train)

Prediction

In [ ]:
yhat = RigeModel.predict(x_test_pr)

Let's compare the first five predicted samples to our test set

In [ ]:
print('predicted:', yhat[0:4])
print('test set :', y_test[0:4].values)

Select the value of Alfa that minimizes the test error, for example, we can use a for loop.

In [ ]:
Rsqu_test = []
Rsqu_train = []
dummy1 = []
ALFA = 10 * np.array(range(0,1000))
for alfa in ALFA:
    RigeModel = Ridge(alpha=alfa) 
    RigeModel.fit(x_train_pr, y_train)
    Rsqu_test.append(RigeModel.score(x_test_pr, y_test))
    Rsqu_train.append(RigeModel.score(x_train_pr, y_train))

Plot out the value of R^2 for different Alphas

In [ ]:
width = 12
height = 10
plt.figure(figsize=(width, height))

plt.plot(ALFA,Rsqu_test, label='validation data  ')
plt.plot(ALFA,Rsqu_train, 'r', label='training Data ')
plt.xlabel('alpha')
plt.ylabel('R^2')
plt.legend()

The blue line represents the R^2 of the test data, and the red line represents the R^2 of the training data. The x-axis represents the different values of Alfa.<br>
The red line in figure represents the  R^2 of the test data, as Alpha increases the R^2 decreases; therefore as Alfa increases the model performs worse on the test data.  The blue line represents the R^2 on the validation data, as the value for Alfa increases the R^2 decreases.

<h3>Grid Search</h3> <br>
The term Alfa is a hyperparameter, sklearn has the class  <b>GridSearchCV</b> to make the process of finding the best hyperparameter simpler.

We create a dictionary of parameter values:

In [ ]:
parameters1= [{'alpha': [0.001,0.1,1, 10, 100, 1000, 10000, 100000, 100000]}]

Create a ridge regions object:

In [ ]:
RR=Ridge()

Create a ridge grid search object

In [ ]:
Grid1 = GridSearchCV(RR, parameters1,cv=4)

Fit the model

In [ ]:
Grid1.fit(x_data[['horsepower', 'curb-weight', 'engine-size', 'highway-mpg']], y_data)

The object finds the best parameter values on the validation data. We can obtain the estimator with the best parameters and assign it to the variable BestRR as follows:

In [ ]:
BestRR=Grid1.best_estimator_
BestRR

We now test the model on the test data

In [ ]:
BestRR.score(x_test[['horsepower', 'curb-weight', 'engine-size', 'highway-mpg']], y_test)